In [1]:
import pandas as pd

In [2]:
npr = pd.read_csv('npr.csv')

In [3]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [4]:
npr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11992 entries, 0 to 11991
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Article  11992 non-null  object
dtypes: object(1)
memory usage: 93.8+ KB


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
#max_df removes words that show up in certain number of documents 0.9 - 90%
count_vect = CountVectorizer(max_df=0.95,min_df=2,stop_words='english')

In [8]:
dtm = count_vect.fit_transform(npr['Article'])

In [10]:
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=7,random_state=42 )

In [11]:
LDA.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

In [14]:
#grab vocabulary
len(count_vect.get_feature_names_out())

54777

In [16]:
type(count_vect.get_feature_names_out())

numpy.ndarray

In [20]:
count_vect.get_feature_names_out()[7888]

'cameras'

In [36]:
import random
random_word_id = random.randint(0,54777)
count_vect.get_feature_names_out()[random_word_id]

'slovenia'

In [39]:
#grab the topics
len(LDA.components_)
#there are 7 topics

7

In [41]:
type(LDA.components_)

numpy.ndarray

In [47]:
LDA.components_.shape
#this LDA component is basically probability distrinution for each 54k words

(7, 54777)

#### this LDA component is basically probability distrinution for each 54k words

In [43]:
single_topic = LDA.components_[0]

In [45]:
single_topic.argsort()
#using this we grab highest probability words
#returns index position in which sorted array will be arranged

array([ 2475, 18302, 35285, ..., 22673, 42561, 42993])

In [46]:
import numpy as np
arr = np.array([10,200,1])
arr.argsort()

array([2, 0, 1])

In [52]:
max(single_topic)

6247.245510521071

In [53]:
min(single_topic)

0.1428571430851871

In [54]:
#grab highest probability of words per topic
single_topic.argsort()[-10:]

array([33390, 36310, 21228, 10425, 31464,  8149, 36283, 22673, 42561,
       42993])

In [58]:
top_20_words = single_topic.argsort()[-20:]

In [59]:
for index in top_20_words:
    print(count_vect.get_feature_names_out()[index])
#something related to health insurance

president
state
tax
insurance
trump
companies
money
year
federal
000
new
percent
government
company
million
care
people
health
said
says


In [62]:
#grab top words for each topic
for index,topic in enumerate(LDA.components_):
    print(f"THE TOP 15 WORDS FOR TOPIC NUM #{index}")
    print([count_vect.get_feature_names_out()[index] for index in topic.argsort()[-15:]])
    print("\n")
    print("\n")

THE TOP 15 WORDS FOR TOPIC NUM #0
['companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']




THE TOP 15 WORDS FOR TOPIC NUM #1
['military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']




THE TOP 15 WORDS FOR TOPIC NUM #2
['way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']




THE TOP 15 WORDS FOR TOPIC NUM #3
['time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']




THE TOP 15 WORDS FOR TOPIC NUM #4
['voters', 'vote', 'election', 'party', 'new', 'obama', 'court', 'republican', 'campaign', 'people', 'state', 'president', 'clinton', 'said', 'trump']




THE TOP 15 WORDS FOR TOPIC NUM #5
['years', 'going', 've', 'life', 'don', 'new', 'way', 'music', 

### now attach these topic number to articles

In [63]:
topic_results = LDA.transform(dtm)

In [67]:
topic_results[0]
#probability of belonging to a topic

array([1.61040465e-02, 6.83341493e-01, 2.25376318e-04, 2.25369288e-04,
       2.99652737e-01, 2.25479379e-04, 2.25497980e-04])

In [68]:
npr['Topic'] = topic_results.argmax(axis=1)

In [70]:
npr.head(10)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1
4,"From photography, illustration and video, to d...",2
5,I did not want to join yoga class. I hated tho...,3
6,With a who has publicly supported the debunk...,3
7,"I was standing by the airport exit, debating w...",2
8,"If movies were trying to be more realistic, pe...",3
9,"Eighteen years ago, on New Year’s Eve, David F...",2
